In [65]:
import string
import re
import random
import collections
import copy

from enum import Enum
Direction = Enum('Direction', 'left right up down upleft upright downleft downright')
    
delta = {Direction.left: (0, -1),Direction.right: (0, 1), 
         Direction.up: (-1, 0), Direction.down: (1, 0), 
         Direction.upleft: (-1, -1), Direction.upright: (-1, 1), 
         Direction.downleft: (1, -1), Direction.downright: (1, 1)}

cat = ''.join
wcat = ' '.join
lcat = '\n'.join

In [66]:
# all_words = [w.strip() for w in open('/usr/share/dict/british-english').readlines()
#             if all(c in string.ascii_lowercase for c in w.strip())]
# words = [w for w in all_words
#          if not any(w in w2 for w2 in all_words if w != w2)]
# open('wordsearch-words', 'w').write(lcat(words))

In [67]:
# ws_words = [w.strip() for w in open('wordsearch-words').readlines()
#             if all(c in string.ascii_lowercase for c in w.strip())]
# ws_words[:10]

In [68]:
ws_words = [w.strip() for w in open('/usr/share/dict/british-english').readlines()
            if all(c in string.ascii_lowercase for c in w.strip())]

In [69]:
def empty_grid(w, h):
    return [['.' for c in range(w)] for r in range(h)]

In [70]:
def show_grid(grid):
    return lcat(cat(r) for r in grid)

In [71]:
grid = empty_grid(10, 10)
print(show_grid(grid))

..........
..........
..........
..........
..........
..........
..........
..........
..........
..........


In [72]:
def indices(grid, r, c, l, d):
    dr, dc = delta[d]
    w = len(grid[0])
    h = len(grid)
    inds = [(r + i * dr, c + i * dc) for i in range(l)]
    return [(i, j) for i, j in inds
           if i >= 0
           if j >= 0
           if i < h
           if j < w]

In [73]:
def gslice(grid, r, c, l, d):
    return [grid[i][j] for i, j in indices(grid, r, c, l, d)]

In [74]:
def set_grid(grid, r, c, d, word):
    for (i, j), l in zip(indices(grid, r, c, len(word), d), word):
        grid[i][j] = l
    return grid

In [75]:
set_grid(grid, 2, 3, Direction.downright, 'testword')
print(show_grid(grid))

..........
..........
...t......
....e.....
.....s....
......t...
.......w..
........o.
.........r
..........


In [76]:
cat(gslice(grid, 3, 2, 15, Direction.right))

'..e.....'

In [77]:
re.match(cat(gslice(grid, 3, 2, 4, Direction.right)), 'keen')

<_sre.SRE_Match object; span=(0, 4), match='keen'>

In [78]:
re.match(cat(gslice(grid, 3, 2, 3, Direction.right)), 'keen')

<_sre.SRE_Match object; span=(0, 3), match='kee'>

In [79]:
re.fullmatch(cat(gslice(grid, 3, 2, 3, Direction.right)), 'keen')

In [80]:
re.match(cat(gslice(grid, 3, 2, 4, Direction.right)), 'kine')

In [81]:
def could_add(grid, r, c, d, word):
    s = gslice(grid, r, c, len(word), d)
    return re.fullmatch(cat(s), word)

In [82]:
could_add(grid, 3, 2, Direction.right, 'keen')

<_sre.SRE_Match object; span=(0, 4), match='keen'>

In [83]:
could_add(grid, 3, 2, Direction.right, 'kine')

In [84]:
random.choice(list(Direction))

<Direction.downleft: 7>

In [129]:
def fill_grid(grid, words, word_count, max_attempts=10000):
    attempts = 0
    added_words = []
    w = len(grid[0])
    h = len(grid)
    while len(added_words) < word_count and attempts < max_attempts:
        attempts += 1
        r = random.randrange(w)
        c = random.randrange(h)
        word = random.choice(words)
        d = random.choice(list(Direction))
        if len(word) >=4 and not any(word in w2 for w2 in added_words) and could_add(grid, r, c, d, word):
            set_grid(grid, r, c, d, word)
            added_words += [word]
            attempts = 0
    return grid, added_words

In [86]:
g = empty_grid(20, 20)
g, ws = fill_grid(g, ws_words, 40)
len(ws)

40

In [87]:
print(show_grid(g))
print(len(ws), 'words added')
print(wcat(ws))

l.fiestasrsnorffas..
a....s..e.a.cawing..
c..gt.dv.re.strongly
i..n..aecmbp....y...
m.eo.uthzoa.of..l.s.
od.lq.esozslhhlyo.k.
ns.e.r.se.ureanoh.r.
o.wby.t.aw.foin.u.u.
ca.o....i.a.to.d.rms
en..l...lerrs.d.i.sk
no...l..i.snalgarn.n
un....a.crappiest.gi
.y.....mdepraved..dw
.mgniggolricochet.ey
.o..pensivelyibmozil
.u.......curd.....fd
.sseitudlevehsid..id
...litchis..romut.ri
.understands......et
....nagilooh......v.
40 words added
understands crappiest archery mallows depraved cawing rawest curd tiny tiddlywinks fiestas zombi duties ricochet uneconomical hope litchis strongly verified logging handing anonymous quaver flours boost holy saffrons errs hooligan male belong tumor dishevel fuzzed raglans pensively murks dents cilia doors


In [88]:
def present(grid, word):
    w = len(grid[0])
    h = len(grid)
    for r in range(h):
        for c in range(w):
            for d in Direction:
                if cat(gslice(grid, r, c, len(word), d)) == word:
                    return True, r, c, d
    return False, 0, 0, list(Direction)[0]

In [89]:
for w in ws:
    print(w, present(g, w))

understands (True, 18, 1, <Direction.right: 2>)
crappiest (True, 11, 8, <Direction.right: 2>)
archery (True, 1, 10, <Direction.downleft: 7>)
mallows (True, 12, 7, <Direction.upleft: 5>)
depraved (True, 12, 8, <Direction.right: 2>)
cawing (True, 1, 12, <Direction.right: 2>)
rawest (True, 9, 11, <Direction.upleft: 5>)
curd (True, 15, 9, <Direction.right: 2>)
tiny (True, 8, 12, <Direction.upright: 6>)
tiddlywinks (True, 18, 19, <Direction.up: 3>)
fiestas (True, 0, 2, <Direction.right: 2>)
zombi (True, 14, 17, <Direction.left: 1>)
duties (True, 16, 7, <Direction.left: 1>)
ricochet (True, 13, 9, <Direction.right: 2>)
uneconomical (True, 11, 0, <Direction.up: 3>)
hope (True, 5, 13, <Direction.upleft: 5>)
litchis (True, 17, 3, <Direction.right: 2>)
strongly (True, 2, 12, <Direction.right: 2>)
verified (True, 19, 18, <Direction.up: 3>)
logging (True, 13, 8, <Direction.left: 1>)
handing (True, 5, 12, <Direction.downright: 8>)
anonymous (True, 8, 1, <Direction.down: 4>)
quaver (True, 5, 4, <Dire

In [125]:
def interesting(grid, words):
    dirs = set(present(grid, w)[3] for w in words)
    return len(words) > 40 and len(dirs) + 1 >= len(delta)

In [126]:
interesting(g, ws)

False

In [131]:
def interesting_grid():
    boring = True
    while boring:
        grid = empty_grid(20, 20)
        grid, words = fill_grid(grid, ws_words, 80)
        boring = not interesting(grid, words)
    return grid, words

In [132]:
g, ws = interesting_grid()
print(show_grid(g))
print(len(ws), 'words added; ', len(set(present(g, w)[3] for w in ws)), 'directions')
print(wcat(ws))

....gnixof...keem...
feihc.spollawvase..s
p.h.shs..snetsafnun.
aeiy.adt..plehdowned
rmcfmzhennaturali.h.
abkake.pteebyelawsay
dlcweln.lnmvrdrawllr
ealnes.s.aeeieslaroe
.zaelreffidclwl...gs
.omtisadeelbst.bg.ei
.noantr...tunet.o.nm
serigamchamoixbemnsb
sd.tnuu..lleterls..e
e.dounf..dekcalsu..s
gyegtcfknobetatser.t
rlkeshskcelf..ploptr
alon.l..sriahdawnsgi
lac..y..gnittilps.od
.eyeball..denedragse
.r..ygnamsecstirg.hs
57 words added;  7 directions
chamoix staunchly keeling wive inns restate settlements byelaws blurt help foxing flecks orals differ unfastens mangy hymens wallops negotiate bestrides largess dawns nobler chief eyeball splitting bleed halogens clamor parade emblazoned hairs meek earmuff slacked retell scented gardened natural grits misery drawl gosh smog stung coked knob tune really secs plop alphas vase downed hazels hick fawn


In [94]:
def datafile(name, sep='\t'):
    """Read key,value pairs from file.
    """
    with open(name) as f:
        for line in f:
            splits = line.split(sep)
            yield [splits[0], int(splits[1])]

In [95]:
def normalise(frequencies):
    """Scale a set of frequencies so they sum to one
    
    >>> sorted(normalise({1: 1, 2: 0}).items())
    [(1, 1.0), (2, 0.0)]
    >>> sorted(normalise({1: 1, 2: 1}).items())
    [(1, 0.5), (2, 0.5)]
    >>> sorted(normalise({1: 1, 2: 1, 3: 1}).items()) # doctest: +ELLIPSIS
    [(1, 0.333...), (2, 0.333...), (3, 0.333...)]
    >>> sorted(normalise({1: 1, 2: 2, 3: 1}).items())
    [(1, 0.25), (2, 0.5), (3, 0.25)]
    """
    length = sum(f for f in frequencies.values())
    return collections.defaultdict(int, ((k, v / length) 
        for (k, v) in frequencies.items()))


In [96]:
english_counts = collections.Counter(dict(datafile('count_1l.txt')))
normalised_english_counts = normalise(english_counts)

In [97]:
wordsearch_counts = collections.Counter(cat(ws_words))
normalised_wordsearch_counts = normalise(wordsearch_counts)

In [118]:
normalised_wordsearch_counts = normalise(collections.Counter(normalised_wordsearch_counts) + collections.Counter({l: 0.05 for l in string.ascii_lowercase}))

In [98]:
def weighted_choice(d):
    """Generate random item from a dictionary of item counts
    """
    target = random.uniform(0, sum(d.values()))
    cuml = 0.0
    for (l, p) in d.items():
        cuml += p
        if cuml > target:
            return l
    return None

def random_english_letter():
    """Generate a random letter based on English letter counts
    """
    return weighted_choice(normalised_english_counts)

def random_wordsearch_letter():
    """Generate a random letter based on wordsearch letter counts
    """
    return weighted_choice(normalised_wordsearch_counts)

In [99]:
cat(sorted(random_english_letter() for i in range(100)))

'aaaaaaaaaabcdddeeeeeeeeeeeefffffgghhhhhhhhhiiiiiiikllmnnnnnnnooooooooprrrrssssssssssssttttttuuvwwwww'

In [100]:
cat(sorted(random_wordsearch_letter() for i in range(100)))

'aaaaaaccccdddeeeeeeeeeeeeeeeeeeeffgghhiiiiikkklllmmmnnnnnnooooooppprrrrrrrrssssssssttttttuuuuuuvwyyy'

In [101]:
random_wordsearch_letter()

'e'

In [102]:
def pad_grid(g0):
    grid = copy.deepcopy(g0)
    w = len(grid[0])
    h = len(grid)
    for r in range(h):
        for c in range(w):
            if grid[r][c] == '.':
                grid[r][c] = random_wordsearch_letter()
    return grid

In [103]:
padded = pad_grid(g)
print(show_grid(padded))

nwtautoimmuneeyinsdl
majorlyerasescmcider
edthrallednxlcawoeaa
gnizeensbnahwwgpsksr
rmisrksiosgiitndtaep
rioigoeopeglbnegsesu
esurnrbdifecihtniust
eeauuieimddlgiiigqan
srcplooscrlufestosve
pdcasmhemaonrgialcel
lguvrepkcrekennronru
ensesmtiesrtiogocwcr
niadpnetulasgpdfeesi
dgthgreoonavhsorinyv
inilpehmnrnntuaeeoae
dioesnmnocstennpolcm
etniwvredwtidnmfdshm
sgsoaarunyyoslurstts
tetoyisimdmaderetlaf
ettflightasnlclquasi


In [104]:
print(show_grid(g))

...autoimmune.......
majorlyerases.m..d..
..thralledn...a..e..
gnizeens..a..wg.sk..
.m.s..si..g.i.ndtae.
.i.ig.eo..gl..egses.
.s.rnrbd..ec.htniust
.eauuiei.ddlg.iigqan
srcploos..lufestosve
p.casmhe.aonrgial.el
lguv.ep.crekennro.ru
ense.m.i.s..iogoc.cr
niad.netulasgp.fee.i
dgt..reo....hs.r.nyv
ini..ehm....t.ae.oa.
dio..nm.o...en.p.lc.
etn.w..e.w..d.....h.
s.so....n.yoslurs.t.
t.t......dmaderetlaf
...flight.s.l..quasi


In [105]:
for w in ws:
    print(w, present(padded, w))

thralled (True, 2, 2, <Direction.right: 2>)
slung (True, 9, 4, <Direction.up: 3>)
freighted (True, 8, 12, <Direction.down: 4>)
townhouse (True, 18, 2, <Direction.upright: 6>)
salute (True, 12, 11, <Direction.left: 1>)
phoebes (True, 10, 6, <Direction.up: 3>)
faltered (True, 18, 19, <Direction.left: 1>)
laywomen (True, 19, 12, <Direction.upleft: 5>)
squeaked (True, 8, 17, <Direction.up: 3>)
perforating (True, 15, 15, <Direction.up: 3>)
iodise (True, 4, 7, <Direction.down: 4>)
lacier (True, 8, 10, <Direction.downleft: 7>)
autoimmune (True, 0, 3, <Direction.right: 2>)
tinging (True, 16, 1, <Direction.up: 3>)
snagged (True, 1, 10, <Direction.down: 4>)
splendidest (True, 8, 0, <Direction.down: 4>)
roughed (True, 10, 9, <Direction.upright: 6>)
crevasse (True, 11, 18, <Direction.up: 3>)
lone (True, 15, 17, <Direction.up: 3>)
ecologists (True, 12, 16, <Direction.up: 3>)
sponge (True, 13, 13, <Direction.up: 3>)
magnetising (True, 1, 14, <Direction.down: 4>)
sneezing (True, 3, 7, <Direction.left

In [141]:
def decoys(grid, words, all_words, limit=100):
    decoy_words = []
    dlen_limit = max(len(w) for w in words)
    while len(words) + len(decoy_words) < limit:
        d = random.choice(all_words)
        if d not in words and len(d) >= 4 and len(d) <= dlen_limit and not present(grid, d)[0]:
            decoy_words += [d]
    return decoy_words

In [135]:
ds = decoys(padded, ws, ws_words)
ds

['incisor',
 'steeled',
 'immobility',
 'undertakings',
 'exhorts',
 'hairnet',
 'placarded',
 'sackful',
 'covenanting',
 'invoking',
 'deltas',
 'nonplus',
 'exactest',
 'eggs',
 'tercentenary',
 'angelic',
 'relearning',
 'ardors',
 'imprints',
 'chamoix',
 'governance',
 'rampart',
 'estuary',
 'poltroons',
 'expect',
 'restaurant',
 'ashrams',
 'illuminates',
 'reprises',
 'seismology',
 'announce',
 'tomorrows',
 'carcinogenics',
 'duplex',
 'transmitters',
 'prosier',
 'anther',
 'masticates',
 'raunchy',
 'briefs',
 'poniard',
 'daunted',
 'topmasts',
 'mynas']

In [108]:
for w in ws + ds:
    print(w, present(padded, w))

thralled (True, 2, 2, <Direction.right: 2>)
slung (True, 9, 4, <Direction.up: 3>)
freighted (True, 8, 12, <Direction.down: 4>)
townhouse (True, 18, 2, <Direction.upright: 6>)
salute (True, 12, 11, <Direction.left: 1>)
phoebes (True, 10, 6, <Direction.up: 3>)
faltered (True, 18, 19, <Direction.left: 1>)
laywomen (True, 19, 12, <Direction.upleft: 5>)
squeaked (True, 8, 17, <Direction.up: 3>)
perforating (True, 15, 15, <Direction.up: 3>)
iodise (True, 4, 7, <Direction.down: 4>)
lacier (True, 8, 10, <Direction.downleft: 7>)
autoimmune (True, 0, 3, <Direction.right: 2>)
tinging (True, 16, 1, <Direction.up: 3>)
snagged (True, 1, 10, <Direction.down: 4>)
splendidest (True, 8, 0, <Direction.down: 4>)
roughed (True, 10, 9, <Direction.upright: 6>)
crevasse (True, 11, 18, <Direction.up: 3>)
lone (True, 15, 17, <Direction.up: 3>)
ecologists (True, 12, 16, <Direction.up: 3>)
sponge (True, 13, 13, <Direction.up: 3>)
magnetising (True, 1, 14, <Direction.down: 4>)
sneezing (True, 3, 7, <Direction.left

In [142]:
g, ws = interesting_grid()
p = pad_grid(g)
ds = decoys(p, ws, ws_words)
print(show_grid(g))
print(show_grid(p))
print(len(ws), 'words added; ', len(set(present(g, w)[3] for w in ws)), 'directions')
print('Present:', wcat(sorted(ws)))
print('Decoys:', wcat(sorted(ds)))

.strigger.essegassum
acselacs.tapri..pgcr
moeclienterr.em.uaie
apisearsclmo.kvpmntp
lebpg..ohlucfaeaespe
ifbi.ev.aafeesr.urol
riae.el.iwfse.o.oqss
evcsr...n..sd.dv..r.
pestdewels..e.aw.ut.
mrlimmersionrl.ob.e.
iyllatnemadnufwls.nl
..sdboomovulesivl.ri
.eiepsreggij.tdeljif
dkwn.atread..oereiat
uais..efile..pnihlhi
rhkripelyt.illsnst.n
iweekendunotablete.g
nfondlyrytsenohsuo..
g.mriffa....naysnp..
.meatspoodle.within.
cstriggerpessegassum
acselacsytapriijpgcr
moeclienterrtemnuaie
apisearsclmookvpmntp
lebpgatohlucfaeaespe
ifbisevxaafeesrlurol
riaehelciwfseioioqss
evcsrkuynpasdfdvetrq
pestdewelsniegawkutd
mrlimmersionrloobuel
iyllatnemadnufwlsanl
dwsdboomovulesivlyri
oeiepsreggijntdeljif
dkwnkatreadvnoereiat
uaiscuefilehapnihlhi
rhkripelytqillsnsten
iweekendunotabletetg
nfondlyrytsenohsuocc
gemriffanternaysnpef
bmeatspoodleswithing
62 words added;  8 directions
Present: adore affirm ages boom burs chain client dens during earmuff feeder file fiver fondly fundamentally hairnet hake honesty 

In [143]:
for i in range(100):
    print(i)
    g, ws = interesting_grid()
    p = pad_grid(g)
    ds = decoys(p, ws, ws_words)
    with open('wordsearch{:02}.txt'.format(i), 'w') as f:
        f.write('20x20\n')
        f.write(show_grid(p))
        f.write('\n')
        f.write(lcat(sorted(ws + ds)))
    with open('wordsearch-solution{:02}.txt'.format(i), 'w') as f:
        f.write('20x20\n')
        f.write(show_grid(g))
        f.write('\n')
        f.write(lcat(sorted(ws)) + '\n\n')
        f.write(lcat(sorted(ds)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
